In [1]:
%load_ext autoreload
%autoreload 2

import json
import os
import random
import sys 
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob
from scipy import interpolate

sys.path.append(os.path.dirname(os.path.abspath('')))
from AutoModelerLoader import AutoModelLoader
from AutoModelerCleaner import AutoModelCleaner
from AutoModelerCluster import AutoModelCluster
from AutoModelerHough import AutoModelHough
from AutoModelerSegmenter import AutoModelSegmenter
from AutoModelerAngleDetector import AutoModelAngleDetector
from execution import execution
from helper import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
from openpyxl import load_workbook as xl_load
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter

# config & data load

In [3]:
with open(os.path.join('..', 'config.json')) as f:
    config = json.load(f)

In [4]:
fn = glob(os.path.join('C:', os.sep, 'Users', 'austinp', 'OneDrive - Synopsys, Inc', 'Data', 'FDC', 'dataset', '*.csv'))
#fn = glob(os.path.join('C:', os.sep, 'Users', 'austinp', 'OneDrive - Synopsys, Inc', 'Data', 'FDC', 'validation_case', '*.csv'))
import re

def extract_number(file_path):
    number = re.search(r'\d+', file_path).group()
    return int(number)

# Sort the list of file paths based on the extracted number
fn = sorted(fn, key=extract_number)

df_list = []
for p in fn:
    df = pd.read_csv(p, dtype={'RECIPE_STEP':str})
    #df = df.iloc[:,1:]
    #df = df.sort_values('Time').reset_index(drop=True)
    df_list.append(df)

# full recipe trace figure save

In [2]:
start_row = 3
start_col = 2

for df, p in zip(df_list, fn):
    df = df.copy()
    pname = p.split(os.sep)[-1].split('.')[0]
    
    loader = AutoModelLoader('LOT_ID', 'WAFER_ID', 'PROCESS', 'PROCESS_STEP', 'RECIPE', 'RECIPE_STEP', 'PARAMETER_NAME', 'PARAMETER_VALUE', 'TIME', df)
    df = loader.data_load()

    # cleaner
    step_list = list(np.sort(np.unique(df.RECIPE_STEP.values).astype(int)).astype('str'))
    cleaner = AutoModelCleaner(config, df)
    cleaned_data, global_stat = cleaner.get_cleaning_data()

    # align all wafer
    reset_data, reset_boundary = reset_index(cleaned_data, step_list)
    
    for rs in step_list:
        plt.figure(figsize=(12,6))
        plt.title(f'{pname}')
        for d in reset_data:
            #print(d.identifier.unique())
            # if d.identifier.unique()[0] == '1922206_1922206.20':
            #     plt.plot(d.PARAMETER_VALUE, marker='o', color='red', linewidth=3, zorder=100)
            plt.plot(d.PARAMETER_VALUE)
        for v in reset_boundary:
            plt.axvline(v, color='black', alpha=0.3)
        #plt.xlim(40, 60)
        #plt.ylim(0, 130)
        max_y = cleaned_data.PARAMETER_VALUE.max()
        for s, v in zip(step_list, reset_boundary):
            plt.axvline(v, color='black', alpha=0.3, linestyle='--')
            plt.text(v-3, max_y, f'{int(s)}')

        image_path = os.path.join('image', 'full_recipe_trace', f'{pname}_step{rs}_{start_row}_{start_col}.png')
        plt.savefig(image_path)
        start_row += 1
        plt.show()

NameError: name 'df_list' is not defined

# original version step trace figure save

In [3]:
start_row = 3
start_col = 5

for df, p in zip(df_list, fn):
    df = df.copy()
    pname = p.split(os.sep)[-1].split('.')[0]
    
    loader = AutoModelLoader('LOT_ID', 'WAFER_ID', 'PROCESS', 'PROCESS_STEP', 'RECIPE', 'RECIPE_STEP', 'PARAMETER_NAME', 'PARAMETER_VALUE', 'TIME', df)
    df = loader.data_load()

    # cleaner
    step_list = list(np.sort(np.unique(df.RECIPE_STEP.values).astype(int)).astype('str'))
    cleaner = AutoModelCleaner(config, df)
    cleaned_data, global_stat = cleaner.get_cleaning_data()
    
    for rs in step_list:
        cluster = AutoModelCluster(cleaned_data, global_stat, recipe_step=rs)
        target_data = cluster.get_clustered_target_data()
        
        segmenter = AutoModelSegmenter(config, global_stat, target_data, 0)

        wafer, seg = segmenter.get_data()
        hough_angle = AutoModelAngleDetector(wafer, seg)
        index_point = hough_angle.get_point()[0]

        print(f'current step:{rs}\n{index_point}')
        plt.figure(figsize=(12,6))
        plt.title(f"{pname} step{rs}\nstart{index_point['start']}, end{index_point['end']}\ntype:{index_point['type']}")
        if index_point['start'] == -1: 
            new_target, _ = segmenter.get_data(is_ignore_undefined_count=False, norm=False)
            for www in new_target:
                plt.plot(www.PARAMETER_VALUE)
        else:
            wafer, _ = segmenter.get_data(norm=False)
            #print(wafer)
            for ww in wafer:
                plt.plot(ww.PARAMETER_VALUE)
                plt.plot(ww.loc[index_point['start']:index_point['end'], 'PARAMETER_VALUE'], color='purple', linewidth='3')
        #plt.show()

        image_path = os.path.join('image', 'original_version_recipe_step', f'{pname}_step{rs}_{start_row}_{start_col}.png')
        plt.savefig(image_path)
        start_row += 1
        plt.show()

NameError: name 'df_list' is not defined

# comparision figure save

In [5]:
# 6번부터 시작

In [ ]:
def create_directory(directory_name):
    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# 예시 사용법
directory_name = "example_directory"
create_directory(directory_name)

In [4]:
dir_name_list = ['ratio_1per', 'ratio_2per', 'ratio_3per', 'ratio_4per']

for k, dir_name in enumerate(dir_name_list):

    if not os.path.exists(os.path.join('image', dir_name)):
        os.makedirs(os.path.join('image', dir_name))
        #rint(f"Directory '{directory_name}' created.")
    else:
        print(f"Directory '{dir_name}' already exists.")
    
    version0 = 0
    version1 = 1 + k

    start_row = 3
    start_col = 6 + k


    for df, p in zip(df_list, fn):
        df = df.copy()
        pname = p.split(os.sep)[-1].split('.')[0]

        loader = AutoModelLoader('LOT_ID', 'WAFER_ID', 'PROCESS', 'PROCESS_STEP', 'RECIPE', 'RECIPE_STEP', 'PARAMETER_NAME', 'PARAMETER_VALUE', 'TIME', df)
        df = loader.data_load()

        # cleaner
        step_list = list(np.sort(np.unique(df.RECIPE_STEP.values).astype(int)).astype('str'))
        cleaner = AutoModelCleaner(config, df)
        cleaned_data, global_stat = cleaner.get_cleaning_data()

        for rs in step_list:
            cluster = AutoModelCluster(cleaned_data, global_stat, recipe_step=rs)
            target_data = cluster.get_clustered_target_data()

            segmenter0 = AutoModelSegmenter(config, global_stat, target_data, version0)
            segmenter1 = AutoModelSegmenter(config, global_stat, target_data, version1)


            wafer0, seg0 = segmenter0.get_data()
            wafer1, seg1 = segmenter1.get_data()

            hough_angle0 = AutoModelAngleDetector(wafer0, seg0)
            hough_angle1 = AutoModelAngleDetector(wafer1, seg1)

            index_point0 = hough_angle0.get_point()[0]
            index_point1 = hough_angle1.get_point()[0]

            if index_point0 != index_point1:
                i = 1
                #print(f"original type:{index_point0['type']}")
                #print(f"new type:{index_point1['type']}")
                index_point_list = [index_point0, index_point1]
                segmenter_list = [segmenter0, segmenter1]
                #wafer_list = [wafer0, wafer1]

                #for i in range(2):
                index_point = index_point_list[i]
                segmenter = segmenter_list[i]
                #wafer = wafer_list[i]

                #fig, ax = plt.subplots(1,3, figsize=(24,5))
                plt.figure(figsize=(12,6))
                plt.title(f"{pname} new version step{rs}\nstart{index_point['start']}, end{index_point['end']}\ntype:{index_point['type']}")
                if index_point['start'] == -1: 
                    new_target, _ = segmenter.get_data(is_ignore_undefined_count=False)
                    for www in new_target:
                        plt.plot(www.PARAMETER_VALUE)
                else:
                    wafer, _ = segmenter.get_data(norm=False)
                    #print(wafer)
                    for ww in wafer:
                        plt.plot(ww.PARAMETER_VALUE)
                        plt.plot(ww.loc[index_point['start']:index_point['end'], 'PARAMETER_VALUE'], color='purple', linewidth='3')
                image_path = os.path.join('image', dir_name, f'{pname}_step{rs}_{start_row}_{start_col}.png')
                plt.savefig(image_path)
                plt.show()


    #                 hough_result = segmenter.result
    #                 hough_result = hough_result[hough_result['type'] != 'UNDEFINED']

    #                 ax[1].set_title(f"new version histogram start")
    #                 ax[1].hist(hough_result.loc[:,'start'], edgecolor='black')

    #                 ax[2].set_title(f"new version histogram end")
    #                 ax[2].hist(hough_result.loc[:,'end'], edgecolor='black')

            else:
                print(f"start:{index_point0['start']}, end:{index_point0['end']}, type:{index_point0['type']}")
                print('no difference\n')
            start_row += 1

NameError: name 'os' is not defined

In [102]:
i = 0
rs = '1'
print(fn[i])

df = df_list[i].copy()
pname = fn[i].split(os.sep)[-1].split('.')[0]
#df = df_list[i].copy()

C:\Users\austinp\OneDrive - Synopsys, Inc\Data\FDC\dataset\case1.csv


In [103]:
loader = AutoModelLoader('LOT_ID', 'WAFER_ID', 'PROCESS', 'PROCESS_STEP', 'RECIPE', 'RECIPE_STEP', 'PARAMETER_NAME', 'PARAMETER_VALUE', 'TIME', df)
df = loader.data_load()

# cleaner
step_list = list(np.sort(np.unique(df.RECIPE_STEP.values).astype(int)).astype('str'))
cleaner = AutoModelCleaner(config, df)
cleaned_data, global_stat = cleaner.get_cleaning_data()

# align all wafer
reset_data, reset_boundary = reset_index(cleaned_data, step_list)

# cluster
cluster = AutoModelCluster(cleaned_data, global_stat, recipe_step=rs)
target_data = cluster.get_clustered_target_data()

In [109]:
image_path

'image\\full_recipe_trace\\case1_rs:1_3_2.png'

In [5]:
start_row = 3
start_col = 2

plt.figure(figsize=(12,6))
plt.title(f'{pname}')
for d in reset_data:
    #print(d.identifier.unique())
    # if d.identifier.unique()[0] == '1922206_1922206.20':
    #     plt.plot(d.PARAMETER_VALUE, marker='o', color='red', linewidth=3, zorder=100)
    plt.plot(d.PARAMETER_VALUE)
for v in reset_boundary:
    plt.axvline(v, color='black', alpha=0.3)
#plt.xlim(40, 60)
#plt.ylim(0, 130)
max_y = cleaned_data.PARAMETER_VALUE.max()
for s, v in zip(step_list, reset_boundary):
    plt.axvline(v, color='black', alpha=0.3, linestyle='--')
    plt.text(v-3, max_y, f'{int(s)}')

image_path = os.path.join('image', 'full_recipe_trace', f'{pname}_step{rs}_{start_row}_{start_col}.png')
plt.savefig(image_path)
plt.show()

NameError: name 'plt' is not defined

In [6]:
for rs in step_list:
    cluster = AutoModelCluster(cleaned_data, global_stat, recipe_step=rs)
    target_data = cluster.get_clustered_target_data()
    #cluster_end = time.time()

    #segmenter_start = time.time()
    segmenter = AutoModelSegmenter(config, global_stat, target_data, 0)


    wafer, seg = segmenter.get_data()

    hough_angle = AutoModelAngleDetector(wafer, seg)

    index_point = hough_angle.get_point()[0]


    #fig, ax = plt.subplots(1,3, figsize=(24,5))
    print(f'current step:{rs}\n{index_point}')
    plt.title(f"{pname} step{rs}\nstart{index_point['start']}, end{index_point['end']}")
    if index_point['start'] == -1: 
        new_target, _ = segmenter.get_data(is_ignore_undefined_count=False, norm=False)
        for www in new_target:
            plt.plot(www.PARAMETER_VALUE)
    else:
        wafer, _ = segmenter.get_data(norm=False)
        #print(wafer)
        for ww in wafer:
            plt.plot(ww.PARAMETER_VALUE)
            plt.plot(ww.loc[index_point['start']:index_point['end'], 'PARAMETER_VALUE'], color='purple', linewidth='3')
    plt.show()
#     hough_result = segmenter.result
#     hough_result = hough_result[hough_result['type'] != 'UNDEFINED']

#     plt.title(f"histogram start")
#     plt.hist(hough_result.loc[:,'start'], edgecolor='black')
#     plt.show()
#     plt.title(f"histogram end")
#     plt.hist(hough_result.loc[:,'end'], edgecolor='black')
#     plt.show()

NameError: name 'step_list' is not defined

# excel save

In [11]:
wb = xl_load('base.xlsx')
ws = wb.active

In [7]:
#image_path = glob(os.path.join('image', 'full_recipe_trace', '*.png'))
#image_path = glob(os.path.join('image', 'original_version_recipe_step', '*.png'))
#image_path = glob(os.path.join('image', 'version1_recipe_step', '*.png'))
#image_path = glob(os.path.join('image', 'version2_recipe_step', '*.png'))
image_path = glob(os.path.join('image', dir_name, '*.png'))

In [10]:
from openpyxl.styles import Alignment

# write case num / recipe step only

In [12]:
for p in image_path:
    case_num = p.split(os.sep)[-1].split('_')[0]
    step = p.split(os.sep)[-1].split('_')[-3]
    row_num = p.split(os.sep)[-1].split('_')[-2]
    col_num = p.split(os.sep)[-1].split('_')[-1].split('.')[0]
    
    row_num = int(row_num)
    #col_num = int(col_num)
    
    # write case_num
    cell_num = f'{get_column_letter(1)}{row_num}'
    ws[cell_num] = case_num

    ws[cell_num].alignment = Alignment(horizontal='center', vertical='center')
    
    # write recipe step
    cell_num = f'{get_column_letter(3)}{row_num}'
    ws[cell_num] = step

    ws[cell_num].alignment = Alignment(horizontal='center', vertical='center')



wb.save('base.xlsx')
wb.close()

# full recipe / original recipe save only

In [16]:
for p in image_path:
    case_num = p.split(os.sep)[-1].split('_')[0]
    step = p.split(os.sep)[-1].split('_')[-3]
    row_num = p.split(os.sep)[-1].split('_')[-2]
    col_num = p.split(os.sep)[-1].split('_')[-1].split('.')[0]
    
    row_num = int(row_num)
    col_num = int(col_num)
    
    img = Image(p)
    #img_width, img_height = img.width, img.height
    img.width, img.height = 506, 272
    
    #excel_col_width = img_width * 0.11
    #excel_row_height = img_height * 0.72
    excel_col_width = 63.36
    excel_row_height = 207.36
    
    # Set the column width and row height to fit the image
    ws.column_dimensions[get_column_letter(col_num)].width = excel_col_width
    ws.row_dimensions[row_num].height = excel_row_height

    # Add the image to the sheet
    img.anchor = f'{get_column_letter(col_num)}{row_num}'  # Example: 'A1'
    ws.add_image(img)
    
    #ws.add_image(img, f'E{start_row}')
    
    #start_row += 1

wb.save('base.xlsx')
wb.close()

# compare elxcel save

In [ ]:
dir_name_list

In [ ]:
image_path = glob(os.path.join('image', dir_name, '*.png'))

In [12]:
for dir_name in dir_name_list:
    
    image_path = glob(os.path.join('image', dir_name, '*.png'))
    row_num_list = []
    
    for p in image_path:
        case_num = p.split(os.sep)[-1].split('_')[0]
        step = p.split(os.sep)[-1].split('_')[-3]
        row_num = p.split(os.sep)[-1].split('_')[-2]
        col_num = p.split(os.sep)[-1].split('_')[-1].split('.')[0]

        row_num = int(row_num)
        col_num = int(col_num)
        row_num_list.append(row_num)

        img = Image(p)
        #img_width, img_height = img.width, img.height
        img.width, img.height = 506, 272

        #excel_col_width = img_width * 0.11
        #excel_row_height = img_height * 0.72
        excel_col_width = 63.36
        excel_row_height = 207.36

        # Set the column width and row height to fit the image
        ws.column_dimensions[get_column_letter(col_num)].width = excel_col_width
        ws.row_dimensions[row_num].height = excel_row_height

        # Add the image to the sheet
        img.anchor = f'{get_column_letter(col_num)}{row_num}'  # Example: 'A1'
        ws.add_image(img)

            #ws.add_image(img, f'E{start_row}')
    for start_row in range(3,174+1):
        cell_num = f'{get_column_letter(col_num)}{start_row}'
        if start_row in row_num_list:
            ws[cell_num] = 'diff'
        else:
            ws[cell_num] = 'same'

        ws[cell_num].alignment = Alignment(horizontal='center', vertical='center')



    wb.save('range_experiment.xlsx')
    wb.close()

In [ ]:
ws['A1'].alignment = Alignment(horizontal='center', vertical='center')